In [1]:
### BENCHMARKING OF THE DIFFERENT PROCEDURES TO CALCULATE A STEERING ANGLE FROM INPUT PICTURES
# Notebook to compare the execution speed of the ResNet, SqueezeNet and OpenCV procedure for 200 testpictures

In [1]:
import torchvision
import torch

device = torch.device('cuda')

# LOAD RESNET
resnet = torchvision.models.resnet18(pretrained=False)
resnet.fc = torch.nn.Linear(512, 1)
resnet.load_state_dict(torch.load('roadFollowing_V2_resnet_conv.pth'))
resnet = resnet.to(device)
resnet = resnet.eval().half()

# LOAD SQUEEZENET
squeezenet = torchvision.models.squeezenet1_1(pretrained=False)
squeezenet.classifier[1] = torch.nn.Conv2d(512, 1, kernel_size=1)
squeezenet.num_classes = 1
squeezenet.load_state_dict(torch.load('roadFollowing_V3_squeeze_conv.pth'))
squeezenet = squeezenet.to(device)
squeezenet = squeezenet.eval().half()

# DEFINE OPEN CV PROCEDURE
import cv2
import numpy as np
import math

def maskImage(image):
    """ Method to define a region of interest with masking an image. 
        In(1): image - The image that should be masked 
        In(2): heightPercent - The percent of the height that should be masked
        Out(1): returns a image where half of the image is masked with a black surface
    """
    height, width = image.shape
    mask = np.zeros_like(image)

    # A polygon defining the region that should be masked as np.array
    polygon = np.array([[
        (0, height * 0.0),  # Up left
        (width, height * 0.0),  # 
        (width, height),
        (0, height),
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(image, mask)

def findCenterline(image):
    slope = 0.0

    # Apply gaussian image blurring
    gaussKernelSize = (3, 3)
    blurred_img = cv2.GaussianBlur(image, gaussKernelSize, 0)
    
    # Convert to HSV image representation
    hsv_img = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Filter out yellow of the center line
    low_yellow = np.array([5, 40, 100])
    up_yellow = np.array([50, 255, 255])
    col_img = cv2.inRange(hsv_img, low_yellow, up_yellow)
    
    # Filter out region of interest
    region_img = maskImage(col_img)
    
    # Apply canny edge detection
    canny_img = cv2.Canny(region_img, 100, 150)
    
    lines = cv2.HoughLinesP(canny_img, rho=1, theta=np.pi/180, threshold=30, lines=np.array([]), minLineLength=5, maxLineGap=50)
    if np.any(lines) == None:
        pass
    else:
        # Center line detection
        lines_x = []
        lines_y = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                lines_x.extend([x1, x2])
                lines_y.extend([y1, y2])

        min_y = int(image.shape[0] * 0.0)
        max_y = image.shape[0]  # <-- The bottom of the image
        
        poly = np.poly1d(np.polyfit(lines_y, lines_x, deg=1))
        center_x_start = int(image.shape[1] * 0.5)  # start in the middle of the picture
        center_x_end = int(poly(min_y))

        if (center_x_end - center_x_start) == 0:
            slope = 0
        else:
            slope = round(np.rad2deg(np.arctan2((max_y - min_y),
                          (center_x_start - center_x_end))) - 90, 3)
    return slope





In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [14]:
import os
import numpy as np
import time

pictures = os.listdir('benchmarkPictures')

def executeSqueezenet(picture):
    start = time.time()
    angle = squeezenet(preprocess(picture)).detach().float().cpu().numpy().flatten()
    angle = np.deg2rad(angle - 90)
    return time.time() - start

def executeResnet(picture):
    start = time.time()
    angle = resnet(preprocess(picture)).detach().float().cpu().numpy().flatten()
    angle = np.deg2rad(angle)
    return time.time() - start

def executeOpenCV(picture):
    start = time.time()
    angle = findCenterline(picture)
    angle = np.deg2rad(angle)
    return time.time() - start

time.sleep(15)

times = []
for paths in pictures:
    image = PIL.Image.open('benchmarkPictures/' + paths)
    pic = np.asarray(image)
    times.append(executeOpenCV(pic))

print("\nBenchmarking result for OpenCV:")
print("---> Mean time to process a picture:  " + str(round(np.mean(times), 6)) + " s      =>    FPS: " + str(1 / np.mean(times)))
print("---> Standard deviation in the time:  " + str(round(np.std(times), 6)))
print("---> Maximum occured processing time: " + str(round(np.max(times), 6))  + " s")

time.sleep(15)

times = []
for paths in pictures:
    image = PIL.Image.open('benchmarkPictures/'+paths)
    pic = np.asarray(image)
    times.append(executeSqueezenet(pic))
    
print("\nBenchmarking result for SQUEEZENET:")
print("---> Mean time to process a picture:  " + str(round(np.mean(times), 6)) + " s      =>    FPS: " + str(1 / np.mean(times)))
print("---> Standard deviation in the time:  " + str(round(np.std(times), 6)))
print("---> Maximum occured processing time: " + str(round(np.max(times), 6))  + " s")

time.sleep(15)

### Do Benchmarking:
times = []
for paths in pictures:
    image = PIL.Image.open('benchmarkPictures/'+paths)
    pic = np.asarray(image)
    times.append(executeResnet(pic))
    
print("\nBenchmarking result for RESNET:")
print("---> Mean time to process a picture:  " + str(round(np.mean(times), 6)) + " s      =>    FPS: " + str(1 / np.mean(times)))
print("---> Standard deviation in the time:  " + str(round(np.std(times), 6)))
print("---> Maximum occured processing time: " + str(round(np.max(times), 6))  + " s")





Benchmarking result for OpenCV:
---> Mean time to process a picture:  0.004404 s      =>    FPS: 227.07684692488556
---> Standard deviation in the time:  0.002817
---> Maximum occured processing time: 0.011779 s

Benchmarking result for SQUEEZENET:
---> Mean time to process a picture:  0.018596 s      =>    FPS: 53.77436915210864
---> Standard deviation in the time:  0.002366
---> Maximum occured processing time: 0.032016 s

Benchmarking result for RESNET:
---> Mean time to process a picture:  0.040098 s      =>    FPS: 24.93858998087455
---> Standard deviation in the time:  0.001371
---> Maximum occured processing time: 0.047792 s
